In [1]:
from pathlib import Path
from os import path as os_path
import pandas as pd
from jinja2 import Template

In [2]:
df = pd.read_excel("ProductsList.xlsx", header=5)

In [3]:
df = df.dropna()

In [4]:
print("Number of MCUs found: {}".format(len(df)))

Number of MCUs found: 48


In [5]:
df.columns=df.columns.str.replace('[^a-zA-Z]', '')
df["svdfile"]=df["PartNumber"].str.slice(0,-2)+".svd"
df["ldscript"]=df["PartNumber"]+"_FLASH.ld"

In [6]:
for q in df["svdfile"].unique():
    print(q)

STM32G030.svd
STM32G031.svd
STM32G041.svd
STM32G070.svd
STM32G071.svd
STM32G081.svd


In [7]:
for q in df["svdfile"].unique():
    exists = os_path.isfile(os_path.join("..","platform-ststm32","misc","svd",q))
    print(exists)

True
True
True
True
True
True


In [8]:
#todo: check if ldfiles exist

In [9]:
template = """{
  "build": {
    "core": "maple",
    "cpu": "cortex-m0plus",
    "extra_flags": "-DSTM32G0xx -D{{PartNumber[:-2]+"xx"}}",
    "f_cpu": "{{ "%.0f"|format(OperatingFrequencyMHz*10)}}00000L",
    "mcu": "{{PartNumber.lower()}}",
    "variant": "stm32g0"
  },
  "debug": {
    "default_tools": [
      "stlink"
    ],
    "jlink_device": "{{PartNumber}}",
    "openocd_target": "stm32g0x",
    "svd_path": "{{svdfile}}"
  },
  "frameworks": [
    "stm32cube"
  ],
  "name": "{{PartNumber}} ({{RAMSizekB|int}}k RAM. {{FlashSizekBProg|int}}k Flash)",
  "upload": {
    "disable_flushing": false,
    "maximum_ram_size": {{RAMSizekB | int*1024}},
    "maximum_size": {{FlashSizekBProg | int*1024}},
    "protocol": "stlink",
    "protocols": [
      "jlink",
      "stlink",
      "blackmagic",
      "serial"
    ],
    "require_upload_port": true,
    "use_1200bps_touch": false,
    "wait_for_upload_port": false
  },
  "url": "https://www.st.com/en/microcontrollers-microprocessors/{{PartNumber.lower()}}.html",
  "vendor": "ST"
}

"""

In [10]:
t = Template(template)
result = t.render(df.iloc[1])
print(result)

{
  "build": {
    "core": "maple",
    "cpu": "cortex-m0plus",
    "extra_flags": "-DSTM32G0xx -DSTM32G030xx",
    "f_cpu": "64000000L",
    "mcu": "stm32g030c8",
    "variant": "stm32g0"
  },
  "debug": {
    "default_tools": [
      "stlink"
    ],
    "jlink_device": "STM32G030C8",
    "openocd_target": "stm32g0x",
    "svd_path": "STM32G030.svd"
  },
  "frameworks": [
    "stm32cube"
  ],
  "name": "STM32G030C8 (8k RAM. 64k Flash)",
  "upload": {
    "disable_flushing": false,
    "maximum_ram_size": 8192,
    "maximum_size": 65536,
    "protocol": "stlink",
    "protocols": [
      "jlink",
      "stlink",
      "blackmagic",
      "serial"
    ],
    "require_upload_port": true,
    "use_1200bps_touch": false,
    "wait_for_upload_port": false
  },
  "url": "https://www.st.com/en/microcontrollers-microprocessors/stm32g030c8.html",
  "vendor": "ST"
}



In [11]:
filename = "generic{{PartNumber}}.json"
output_folder = "output"

In [12]:
ft = Template(filename)

In [13]:
Path(output_folder).mkdir(exist_ok=True)

In [15]:
for _,row in df.iterrows():
    with open(os_path.join(output_folder,ft.render(row)), "w") as f:
        f.write(t.render(row))

In [ ]:
df